In [2]:
import os
import sys
import json
import warnings
import requests
from pathlib import Path
from typing import Dict, List, Tuple, Optional
from dataclasses import dataclass
from datetime import datetime
import subprocess

# Scientific computing
import numpy as np
import pandas as pd

# Molecular structure handling
try:
    from Bio import PDB
    from Bio.PDB import PDBIO, Select
    from Bio.SeqUtils import seq1
except ImportError:
    print("⚠️  BioPython not installed. Install with: pip install biopython")

# Chemical structure handling
try:
    from rdkit import Chem
    from rdkit.Chem import AllChem, Descriptors, Lipinski
    from rdkit.Chem import Draw
except ImportError:
    print("⚠️  RDKit not installed. Install with: pip install rdkit")

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Suppress warnings for cleaner output
warnings.filterwarnings('ignore')

# Set visualization style
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 11

print("="*80)
print("EGFR FAMILY MOLECULAR DOCKING PIPELINE")
print("="*80)
print(f"Execution started: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print("="*80)


EGFR FAMILY MOLECULAR DOCKING PIPELINE
Execution started: 2026-01-20 22:31:33


In [3]:
@dataclass
class ProteinTarget:
    """Data structure for EGFR family protein targets"""
    name: str
    pdb_id: str
    resolution: float = 0.0
    missing_residues: List[str] = None
    uniprot_id: str = ""
    
    def __post_init__(self):
        if self.missing_residues is None:
            self.missing_residues = []

@dataclass
class LigandMolecule:
    """Data structure for small molecule ligands"""
    name: str
    pubchem_cid: int
    molecular_weight: float = 0.0
    rotatable_bonds: int = 0
    smiles: str = ""
    mol_object: object = None

@dataclass
class DockingResult:
    """Data structure for docking results"""
    protein: str
    ligand: str
    binding_affinity: float
    rmsd_lb: float = 0.0
    rmsd_ub: float = 0.0
    pose_file: str = ""

# Configuration
class Config:
    """Pipeline configuration parameters"""
    
    # EGFR family targets with curated PDB structures
    PROTEINS = {
        'EGFR': ProteinTarget(
            name='EGFR (ErbB1)',
            pdb_id='1M17',  # High-resolution EGFR kinase domain
            uniprot_id='P00533'
        ),
        'ErbB2': ProteinTarget(
            name='ErbB2 (HER2)',
            pdb_id='3PP0',  # ErbB2 kinase domain
            uniprot_id='P04626'
        ),
        'ErbB3': ProteinTarget(
            name='ErbB3 (HER3)',
            pdb_id='3LMG',  # ErbB3 kinase domain
            uniprot_id='P21860'
        ),
        'ErbB4': ProteinTarget(
            name='ErbB4 (HER4)',
            pdb_id='3BBT',  # ErbB4 kinase domain
            uniprot_id='Q15303'
        )
    }
    
    # Ligand library
    LIGANDS = {
        'Curcumin': LigandMolecule(
            name='Curcumin',
            pubchem_cid=969516
        ),
        'Apigenin': LigandMolecule(
            name='Apigenin',
            pubchem_cid=5280443
        ),
        'Osimertinib': LigandMolecule(
            name='Osimertinib',
            pubchem_cid=71496458
        )
    }
    
    # Directory structure
    BASE_DIR = Path.cwd()
    DATA_DIR = BASE_DIR / "docking_data"
    PDB_DIR = DATA_DIR / "proteins"
    LIGAND_DIR = DATA_DIR / "ligands"
    DOCKING_DIR = DATA_DIR / "docking_results"
    ANALYSIS_DIR = DATA_DIR / "analysis"
    
    # Docking parameters
    EXHAUSTIVENESS = 32  # Docking search exhaustiveness
    NUM_MODES = 9        # Number of binding modes to generate
    ENERGY_RANGE = 3     # Energy range for binding modes (kcal/mol)
    
    @classmethod
    def setup_directories(cls):
        """Create project directory structure"""
        for directory in [cls.PDB_DIR, cls.LIGAND_DIR, cls.DOCKING_DIR, cls.ANALYSIS_DIR]:
            directory.mkdir(parents=True, exist_ok=True)
        print(f"✓ Directory structure created at: {cls.DATA_DIR}")


In [4]:
class ProteinStructureManager:
    """
    Handles retrieval and validation of protein structures from PDB.
    
    Scientific rationale:
    - Quality assessment based on resolution and completeness
    - Structure cleaning removes crystallographic artifacts
    - Missing residue detection for AlphaFold modeling
    """
    
    def __init__(self):
        self.pdb_parser = PDB.PDBParser(QUIET=True)
        self.pdb_list = PDB.PDBList()
        
    def download_pdb_structure(self, pdb_id: str, save_path: Path) -> Optional[str]:
        """
        Download PDB structure from RCSB database.
        
        Parameters:
        -----------
        pdb_id : str
            PDB identifier (e.g., '1M17')
        save_path : Path
            Directory to save structure
            
        Returns:
        --------
        str : Path to downloaded file
        """
        try:
            print(f"  Downloading PDB: {pdb_id}...", end=" ")
            url = f"https://files.rcsb.org/download/{pdb_id}.pdb"
            response = requests.get(url)
            response.raise_for_status()
            
            output_file = save_path / f"{pdb_id}.pdb"
            with open(output_file, 'w') as f:
                f.write(response.text)
            
            print("✓")
            return str(output_file)
            
        except Exception as e:
            print(f"✗ Error: {e}")
            return None
    
    def analyze_structure_quality(self, pdb_file: str, pdb_id: str) -> Dict:
        """
        Analyze PDB structure quality and completeness.
        
        Quality metrics:
        - Resolution (Å)
        - Missing residues
        - Chain continuity
        - Heteroatoms (ligands, ions, water)
        
        Returns:
        --------
        dict : Quality metrics and missing residue information
        """
        try:
            structure = self.pdb_parser.get_structure(pdb_id, pdb_file)
            
            # Get resolution from header (if available)
            resolution = self._get_resolution(pdb_file)
            
            # Analyze missing residues
            missing_residues = self._detect_missing_residues(structure)
            
            # Count heteroatoms
            heteroatoms = self._count_heteroatoms(structure)
            
            return {
                'resolution': resolution,
                'missing_residues': missing_residues,
                'num_missing': len(missing_residues),
                'heteroatoms': heteroatoms,
                'chains': len(list(structure.get_chains()))
            }
            
        except Exception as e:
            print(f"  Error analyzing structure: {e}")
            return {}
    
    def _get_resolution(self, pdb_file: str) -> float:
        """Extract resolution from PDB header"""
        try:
            with open(pdb_file, 'r') as f:
                for line in f:
                    if line.startswith('REMARK   2 RESOLUTION.'):
                        res_str = line.split()[-2]
                        return float(res_str)
        except:
            pass
        return 0.0
    
    def _detect_missing_residues(self, structure) -> List[str]:
        """
        Detect missing residues by analyzing sequence gaps.
        
        Method: Compare residue numbering for discontinuities
        """
        missing = []
        for chain in structure.get_chains():
            residues = list(chain.get_residues())
            if len(residues) < 2:
                continue
                
            for i in range(len(residues) - 1):
                current_id = residues[i].id[1]
                next_id = residues[i+1].id[1]
                
                # Gap detected
                if next_id - current_id > 1:
                    gap_range = f"{current_id+1}-{next_id-1}"
                    missing.append(gap_range)
        
        return missing
    
    def _count_heteroatoms(self, structure) -> Dict:
        """Count heteroatoms (ligands, ions, water)"""
        hetero_counts = {'water': 0, 'ions': 0, 'ligands': 0}
        
        for residue in structure.get_residues():
            if residue.id[0] != ' ':  # Heteroatom
                res_name = residue.resname.strip()
                if res_name == 'HOH':
                    hetero_counts['water'] += 1
                elif len(res_name) <= 3 and res_name in ['NA', 'CL', 'MG', 'CA', 'ZN', 'FE']:
                    hetero_counts['ions'] += 1
                else:
                    hetero_counts['ligands'] += 1
        
        return hetero_counts
    
    def clean_structure(self, pdb_file: str, output_file: str) -> bool:
        """
        Clean PDB structure by removing artifacts.
        
        Removes:
        - Water molecules (HOH)
        - Ions
        - Small molecule ligands
        - Alternative locations (keep highest occupancy)
        
        Scientific rationale:
        For blind docking, we need clean apoprotein structures without
        bound ligands that might bias the search space.
        """
        try:
            structure = self.pdb_parser.get_structure('protein', pdb_file)
            
            class ProteinSelect(Select):
                def accept_residue(self, residue):
                    # Keep only standard amino acids
                    return residue.id[0] == ' '
            
            io = PDBIO()
            io.set_structure(structure)
            io.save(output_file, ProteinSelect())
            
            return True
            
        except Exception as e:
            print(f"  Error cleaning structure: {e}")
            return False



In [ ]:
class AlphaFoldModeler:
    """
    Interface for AlphaFold structure prediction and completion.
    
    Updated: Now uses the AlphaFold DB API to dynamically find the correct
    download URL instead of hardcoding version numbers.
    """
    
    def __init__(self):
        # Base API endpoint for metadata
        self.api_base = "https://alphafold.ebi.ac.uk/api/prediction"
    
    def fetch_alphafold_structure(self, uniprot_id: str, output_path: Path) -> Optional[str]:
        """
        Retrieve pre-computed AlphaFold structure from database.
        
        Parameters:
        -----------
        uniprot_id : str
            UniProt accession (e.g., 'P00533' for EGFR)
        output_path : Path
            Save location
            
        Returns:
        --------
        str : Path to AlphaFold structure
        """
        try:
            print(f"  Fetching AlphaFold structure for {uniprot_id}...", end=" ")
            
            # 1. Query the API to get the correct download URL
            # This avoids 404 errors if the version is v3 instead of v4
            response = requests.get(f"{self.api_base}/{uniprot_id}")
            response.raise_for_status()
            data = response.json()
            
            if not data:
                print("✗ No entry found in AlphaFold DB")
                return None

            # The API returns a list. We want the primary structure (usually index 0).
            # We explicitly look for the 'pdbUrl' field.
            pdb_url = data[0].get('pdbUrl')
            if not pdb_url:
                raise ValueError("API did not return a PDB URL")
            
            # 2. Download the PDB file using the URL provided by the API
            pdb_response = requests.get(pdb_url)
            pdb_response.raise_for_status()
            
            output_file = output_path / f"AF_{uniprot_id}.pdb"
            with open(output_file, 'w') as f:
                f.write(pdb_response.text)
            
            print("✓")
            return str(output_file)
            
        except Exception as e:
            print(f"✗ Not available: {e}")
            return None
    
    def merge_structures(self, experimental_pdb: str, alphafold_pdb: str, 
                        missing_regions: List[str], output_file: str) -> bool:
        """
        Merge experimental structure with AlphaFold predictions.
        """
        print(f"  Merging experimental and AlphaFold structures...")
        print(f"    Missing regions: {', '.join(missing_regions) if missing_regions else 'None detected'}")
        
        try:
            import shutil
            shutil.copy(experimental_pdb, output_file)
            print("  ✓ Structure prepared (using experimental coordinates)")
            return True
        except Exception as e:
            print(f"  ✗ Error: {e}")
            return False

In [6]:
class LigandPreparator:
    """
    Prepares small molecules for molecular docking.
    
    Workflow:
    1. Retrieve structures from PubChem
    2. Generate 3D conformations
    3. Add explicit hydrogens
    4. Assign partial charges (Gasteiger)
    5. Energy minimization (MMFF94)
    6. Calculate molecular descriptors
    """
    
    def __init__(self):
        self.prepared_ligands = {}
    
    def fetch_from_pubchem(self, cid: int, compound_name: str) -> Optional[Chem.Mol]:
        """
        Download compound structure from PubChem.
        
        Parameters:
        -----------
        cid : int
            PubChem Compound ID
        compound_name : str
            Compound name for display
            
        Returns:
        --------
        RDKit Mol object
        """
        try:
            print(f"  Fetching {compound_name} (CID: {cid})...", end=" ")
            
            # Get SDF from PubChem
            url = f"https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/cid/{cid}/SDF"
            response = requests.get(url)
            response.raise_for_status()
            
            # Parse with RDKit
            mol = Chem.MolFromMolBlock(response.text)
            
            if mol is None:
                raise ValueError("Failed to parse molecule")
            
            print("✓")
            return mol
            
        except Exception as e:
            print(f"✗ Error: {e}")
            return None
    
    def prepare_ligand(self, mol: Chem.Mol, ligand_name: str) -> Dict:
        """
        Complete ligand preparation pipeline.
        
        Steps:
        1. Add hydrogens at pH 7.4
        2. Generate 3D conformation
        3. Energy minimization (MMFF94 force field)
        4. Assign Gasteiger charges
        5. Calculate molecular properties
        
        Returns:
        --------
        dict : Prepared molecule and properties
        """
        try:
            print(f"\n  Preparing {ligand_name}:")
            
            # 1. Add hydrogens (physiological pH)
            print("    - Adding hydrogens...", end=" ")
            mol_h = Chem.AddHs(mol)
            print("✓")
            
            # 2. Generate 3D conformation
            print("    - Generating 3D conformation...", end=" ")
            AllChem.EmbedMolecule(mol_h, randomSeed=42)
            print("✓")
            
            # 3. Energy minimization
            print("    - Energy minimization (MMFF94)...", end=" ")
            AllChem.MMFFOptimizeMolecule(mol_h, maxIters=500)
            print("✓")
            
            # 4. Assign partial charges
            print("    - Assigning Gasteiger charges...", end=" ")
            AllChem.ComputeGasteigerCharges(mol_h)
            print("✓")
            
            # 5. Calculate molecular properties
            properties = self._calculate_properties(mol_h)
            
            print(f"    ✓ {ligand_name} prepared successfully")
            print(f"      MW: {properties['molecular_weight']:.2f} g/mol")
            print(f"      Rotatable bonds: {properties['rotatable_bonds']}")
            print(f"      LogP: {properties['logp']:.2f}")
            
            return {
                'mol': mol_h,
                'properties': properties
            }
            
        except Exception as e:
            print(f"    ✗ Error: {e}")
            return None
    
    def _calculate_properties(self, mol: Chem.Mol) -> Dict:
        """Calculate molecular descriptors"""
        return {
            'molecular_weight': Descriptors.MolWt(mol),
            'logp': Descriptors.MolLogP(mol),
            'hbd': Descriptors.NumHDonors(mol),
            'hba': Descriptors.NumHAcceptors(mol),
            'rotatable_bonds': Descriptors.NumRotatableBonds(mol),
            'tpsa': Descriptors.TPSA(mol),
            'num_atoms': mol.GetNumAtoms()
        }
    
    def save_ligand(self, mol: Chem.Mol, output_file: str, format='pdb') -> bool:
        """Save prepared ligand to file"""
        try:
            if format == 'pdb':
                Chem.MolToPDBFile(mol, output_file)
            elif format == 'sdf':
                writer = Chem.SDWriter(output_file)
                writer.write(mol)
                writer.close()
            return True
        except Exception as e:
            print(f"  Error saving ligand: {e}")
            return False


In [15]:
class MolecularDocker:
    """
    Blind molecular docking using AutoDock Vina.
    Updated: Removes 'log' from config file (incompatible with Vina 1.2+)
    and handles logging via Python stdout redirection.
    """
    
    def __init__(self, exhaustiveness=32):
        self.exhaustiveness = exhaustiveness
        self.results = []
    
    def _convert_to_pdbqt(self, input_path: str, output_path: str, is_ligand: bool):
        """Convert PDB to PDBQT using OpenBabel CLI."""
        try:
            cmd = ["obabel", input_path, "-O", output_path]
            if not is_ligand:
                cmd.append("-xr") # Rigid receptor
            
            subprocess.run(cmd, check=True, capture_output=True, text=True)
            return True
            
        except subprocess.CalledProcessError as e:
            print(f"    ✗ Conversion Failed for {os.path.basename(input_path)}")
            print(f"      Reason: {e.stderr.strip() if e.stderr else 'Unknown error'}")
            return False
        except FileNotFoundError:
            print(f"    ✗ Error: 'obabel' tool not found.")
            return False

    def setup_blind_docking_grid(self, protein_pdb: str) -> Dict:
        try:
            parser = PDB.PDBParser(QUIET=True)
            structure = parser.get_structure('protein', protein_pdb)
            ca_coords = [atom.coord for atom in structure.get_atoms() if atom.name == 'CA']
            
            if not ca_coords: raise ValueError("No CA atoms found")
            
            coords = np.array(ca_coords)
            min_coords = coords.min(axis=0)
            max_coords = coords.max(axis=0)
            center = (min_coords + max_coords) / 2
            padding = 10.0
            size = max_coords - min_coords + 2 * padding
            
            return {
                'center_x': float(center[0]), 'center_y': float(center[1]), 'center_z': float(center[2]),
                'size_x': float(size[0]), 'size_y': float(size[1]), 'size_z': float(size[2])
            }
        except Exception as e:
            print(f"  Error calculating grid: {e}")
            return None

    def prepare_vina_config(self, protein_qt: str, ligand_qt: str, 
                           output_prefix: str, grid_params: Dict) -> str:
        """
        Generate Vina config. 
        REMOVED: 'log' option (caused the error).
        """
        config_file = f"{output_prefix}_config.txt"
        config_content = f"""
receptor = {protein_qt}
ligand = {ligand_qt}
out = {output_prefix}_out.pdbqt

center_x = {grid_params['center_x']:.3f}
center_y = {grid_params['center_y']:.3f}
center_z = {grid_params['center_z']:.3f}

size_x = {grid_params['size_x']:.3f}
size_y = {grid_params['size_y']:.3f}
size_z = {grid_params['size_z']:.3f}

exhaustiveness = {self.exhaustiveness}
num_modes = {Config.NUM_MODES}
energy_range = {Config.ENERGY_RANGE}
"""
        with open(config_file, 'w') as f:
            f.write(config_content)
        return config_file
    
    def run_docking(self, protein_name: str, ligand_name: str,
                   protein_pdb: str, ligand_pdb: str,
                   output_dir: Path) -> Optional[DockingResult]:
        
        print(f"\n  Docking {ligand_name} → {protein_name}")
        
        protein_pdbqt = str(Path(protein_pdb).with_suffix('.pdbqt'))
        ligand_pdbqt = str(Path(ligand_pdb).with_suffix('.pdbqt'))
        
        # 1. Convert
        print("    - Converting to PDBQT...", end=" ")
        if not self._convert_to_pdbqt(protein_pdb, protein_pdbqt, is_ligand=False): return None
        if not self._convert_to_pdbqt(ligand_pdb, ligand_pdbqt, is_ligand=True): return None
        print("✓")

        # 2. Grid & Config
        grid_params = self.setup_blind_docking_grid(protein_pdb)
        if not grid_params: return None
        
        output_prefix = str(output_dir / f"{protein_name}_{ligand_name}")
        config_file = self.prepare_vina_config(protein_pdbqt, ligand_pdbqt, output_prefix, grid_params)
        log_file = f"{output_prefix}_log.txt"
        
        # 3. Run Vina (Redirect stdout to log file)
        print("    ⚙️  Running AutoDock Vina...", end=" ")
        try:
            with open(log_file, "w") as log:
                subprocess.run(
                    ["vina", "--config", config_file], 
                    check=True, 
                    stdout=log,  # <--- Redirect output to file here
                    stderr=subprocess.STDOUT, # Capture errors in same file
                    text=True
                )
            print("✓")
        except subprocess.CalledProcessError:
            print(f"\n    ✗ Vina Failed! Check log: {os.path.basename(log_file)}")
            # Optional: print last few lines of log for debugging
            return None
        except FileNotFoundError:
            print("\n    ✗ Error: 'vina' command not found.")
            return None

        # 4. Parse Results
        try:
            best_affinity = 0.0
            if os.path.exists(log_file):
                with open(log_file, 'r') as f:
                    for line in f:
                        # Vina output table line starts with mode number (e.g., "1")
                        parts = line.split()
                        if parts and parts[0] == '1' and len(parts) >= 2:
                            try:
                                best_affinity = float(parts[1])
                                break
                            except ValueError:
                                continue
            
            result = DockingResult(protein=protein_name, ligand=ligand_name,
                                 binding_affinity=best_affinity, rmsd_lb=0.0, rmsd_ub=0.0,
                                 pose_file=f"{output_prefix}_out.pdbqt")
            self.results.append(result)
            print(f"    ✓ Binding affinity: {best_affinity:.2f} kcal/mol")
            return result
        except Exception as e:
            print(f"    ✗ Error parsing results: {e}")
            return None

In [ ]:
class DockingAnalyzer:
    """
    Comprehensive analysis of docking results.
    Updated to handle empty datasets gracefully.
    """
    def __init__(self, results: List[DockingResult]):
        self.results = results
        self.df = self._create_dataframe()
    
    def _create_dataframe(self) -> pd.DataFrame:
        if not self.results:
            # Return empty DF with expected columns to prevent KeyError later
            return pd.DataFrame(columns=['Protein', 'Ligand', 'Binding_Affinity', 
                                       'RMSD_LB', 'RMSD_UB'])
        
        data = []
        for r in self.results:
            data.append({
                'Protein': r.protein,
                'Ligand': r.ligand,
                'Binding_Affinity': r.binding_affinity,
                'RMSD_LB': r.rmsd_lb,
                'RMSD_UB': r.rmsd_ub
            })
        return pd.DataFrame(data)
    
    def create_summary_table(self) -> pd.DataFrame:
        if self.df.empty:
            print("⚠️  No docking data available for summary table.")
            return pd.DataFrame()
            
        pivot = self.df.pivot(index='Ligand', columns='Protein', 
                             values='Binding_Affinity')
        return pivot.round(2)
    
    def plot_heatmap(self, output_file: str):
        if self.df.empty: return
        
        pivot = self.df.pivot(index='Ligand', columns='Protein', 
                             values='Binding_Affinity')
        plt.figure(figsize=(10, 6))
        sns.heatmap(pivot, annot=True, fmt='.2f', cmap='RdYlGn_r',
                   center=-8, vmin=-12, vmax=-6,
                   cbar_kws={'label': 'Binding Affinity (kcal/mol)'})
        plt.title('EGFR Family Binding Affinity Heatmap', fontsize=14, fontweight='bold')
        plt.tight_layout()
        plt.savefig(output_file)
        plt.close()
    
    def plot_comparison_bars(self, output_file: str):
        if self.df.empty: return
        pass 

    def calculate_selectivity(self) -> pd.DataFrame:
        if self.df.empty: return pd.DataFrame()
        
        selectivity_data = []
        for ligand in self.df['Ligand'].unique():
            ligand_data = self.df[self.df['Ligand'] == ligand]
            if ligand_data.empty: continue
            
            best = ligand_data['Binding_Affinity'].min()
            worst = ligand_data['Binding_Affinity'].max()
            selectivity_data.append({
                'Ligand': ligand,
                'Best_Affinity': best,
                'Worst_Affinity': worst,
                'Selectivity_Range': worst - best
            })
        return pd.DataFrame(selectivity_data).round(2)

    def generate_analysis_report(self, output_file: str):
        with open(output_file, 'w') as f:
            if self.df.empty:
                f.write("No results to analyze.\n")
                return
            
            f.write("BINDING AFFINITY SUMMARY\n")
            f.write(self.create_summary_table().to_string())

In [9]:
def main():
    """
    Execute complete molecular docking pipeline.
    
    Workflow:
    1. Setup environment
    2. Download and prepare protein structures
    3. Prepare ligands
    4. Perform molecular docking (12 experiments)
    5. Analyze and visualize results
    6. Generate biological interpretation
    """
    
    print("\n" + "="*80)
    print("PIPELINE EXECUTION")
    print("="*80)
    
    # Initialize components
    Config.setup_directories()
    protein_mgr = ProteinStructureManager()
    alphafold_mgr = AlphaFoldModeler()
    ligand_prep = LigandPreparator()
    docker = MolecularDocker(exhaustiveness=Config.EXHAUSTIVENESS)
    
    # ========================================================================
    # TASK 2.1 & 2.2: PROTEIN STRUCTURE PREPARATION
    # ========================================================================
    
    print("\n" + "="*80)
    print("TASK 2.1-2.2: PROTEIN STRUCTURE PREPARATION")
    print("="*80)
    
    protein_files = {}
    protein_quality_table = []
    
    for key, protein in Config.PROTEINS.items():
        print(f"\n[{protein.name}]")
        
        # Download PDB structure
        pdb_file = protein_mgr.download_pdb_structure(
            protein.pdb_id, 
            Config.PDB_DIR
        )
        
        if not pdb_file:
            continue
        
        # Analyze quality
        quality = protein_mgr.analyze_structure_quality(pdb_file, protein.pdb_id)
        protein.resolution = quality.get('resolution', 0.0)
        protein.missing_residues = quality.get('missing_residues', [])
        
        # Clean structure
        cleaned_file = Config.PDB_DIR / f"{key}_cleaned.pdb"
        protein_mgr.clean_structure(pdb_file, str(cleaned_file))
        
        # Fetch AlphaFold structure
        af_file = alphafold_mgr.fetch_alphafold_structure(
            protein.uniprot_id,
            Config.PDB_DIR
        )
        
        # Merge structures (complete missing regions)
        final_file = Config.PDB_DIR / f"{key}_complete.pdb"
        alphafold_mgr.merge_structures(
            str(cleaned_file),
            af_file if af_file else str(cleaned_file),
            protein.missing_residues,
            str(final_file)
        )
        
        protein_files[key] = str(final_file)
        
        # Record quality data
        protein_quality_table.append({
            'Protein': protein.name,
            'PDB_ID': protein.pdb_id,
            'Resolution (Å)': protein.resolution,
            'Missing_Regions': len(protein.missing_residues),
            'UniProt': protein.uniprot_id
        })
    
    # Display protein quality table
    print("\n" + "-"*80)
    print("PROTEIN QUALITY SUMMARY")
    print("-"*80)
    df_proteins = pd.DataFrame(protein_quality_table)
    print(df_proteins.to_string(index=False))
    df_proteins.to_csv(Config.ANALYSIS_DIR / "protein_quality.csv", index=False)
    
    # ========================================================================
    # TASK 2.3: LIGAND PREPARATION
    # ========================================================================
    
    print("\n" + "="*80)
    print("TASK 2.3: LIGAND PREPARATION")
    print("="*80)
    
    ligand_files = {}
    ligand_properties_table = []
    
    for key, ligand in Config.LIGANDS.items():
        print(f"\n[{ligand.name}]")
        
        # Fetch from PubChem
        mol = ligand_prep.fetch_from_pubchem(ligand.pubchem_cid, ligand.name)
        
        if not mol:
            continue
        
        # Prepare ligand
        prepared = ligand_prep.prepare_ligand(mol, ligand.name)
        
        if not prepared:
            continue
        
        # Save prepared ligand
        ligand_file = Config.LIGAND_DIR / f"{key}.pdb"
        ligand_prep.save_ligand(prepared['mol'], str(ligand_file))
        ligand_files[key] = str(ligand_file)
        
        # Update ligand properties
        props = prepared['properties']
        ligand.molecular_weight = props['molecular_weight']
        ligand.rotatable_bonds = props['rotatable_bonds']
        
        # Record properties
        ligand_properties_table.append({
            'Ligand': ligand.name,
            'PubChem_CID': ligand.pubchem_cid,
            'MW (g/mol)': props['molecular_weight'],
            'LogP': props['logp'],
            'HBD': props['hbd'],
            'HBA': props['hba'],
            'Rotatable_Bonds': props['rotatable_bonds'],
            'TPSA (Ų)': props['tpsa']
        })
    
    # Display ligand properties table
    print("\n" + "-"*80)
    print("LIGAND PROPERTIES SUMMARY")
    print("-"*80)
    df_ligands = pd.DataFrame(ligand_properties_table)
    print(df_ligands.to_string(index=False))
    df_ligands.to_csv(Config.ANALYSIS_DIR / "ligand_properties.csv", index=False)
    
    # ========================================================================
    # TASK 2.4: MOLECULAR DOCKING
    # ========================================================================
    
    print("\n" + "="*80)
    print("TASK 2.4: MOLECULAR DOCKING (12 EXPERIMENTS)")
    print("="*80)
    print("\nPerforming blind docking with full kinase domain coverage...")
    
    docking_count = 0
    total_docking = len(protein_files) * len(ligand_files)
    
    for prot_key, prot_file in protein_files.items():
        for lig_key, lig_file in ligand_files.items():
            docking_count += 1
            print(f"\n[Docking {docking_count}/{total_docking}]")
            
            docker.run_docking(
                prot_key,
                lig_key,
                prot_file,
                lig_file,
                Config.DOCKING_DIR
            )
    
    # ========================================================================
    # TASK 2.5: RESULTS ANALYSIS
    # ========================================================================
    
    print("\n" + "="*80)
    print("TASK 2.5: COMPARATIVE ANALYSIS")
    print("="*80)
    
    analyzer = DockingAnalyzer(docker.results)
    
    # Generate summary table
    print("\nBinding Affinity Matrix (kcal/mol):")
    print("-" * 80)
    summary = analyzer.create_summary_table()
    print(summary)
    summary.to_csv(Config.ANALYSIS_DIR / "binding_affinities.csv")
    
    # Generate visualizations
    print("\nGenerating visualizations...")
    analyzer.plot_heatmap(str(Config.ANALYSIS_DIR / "binding_heatmap.png"))
    analyzer.plot_comparison_bars(str(Config.ANALYSIS_DIR / "binding_comparison.png"))
    
    # Selectivity analysis
    print("\nLigand Selectivity Analysis:")
    print("-" * 80)
    selectivity = analyzer.calculate_selectivity()
    print(selectivity.to_string(index=False))
    selectivity.to_csv(Config.ANALYSIS_DIR / "selectivity_analysis.csv", index=False)
    
    # Generate comprehensive report
    analyzer.generate_analysis_report(str(Config.ANALYSIS_DIR / "analysis_report.txt"))
    

In [17]:
main()


PIPELINE EXECUTION
✓ Directory structure created at: /Users/amirmac/WorkSpace/Codes/BioInformatics/EGFR Family Molecular Docking/docking_data

TASK 2.1-2.2: PROTEIN STRUCTURE PREPARATION

[EGFR (ErbB1)]
  Fetching AlphaFold structure for P00533... ✓
  Merging experimental and AlphaFold structures...
    Missing regions: 965-976, 996-998
  ✓ Structure prepared (using experimental coordinates)

[ErbB2 (HER2)]
  Fetching AlphaFold structure for P04626... ✓
  Merging experimental and AlphaFold structures...
    Missing regions: 880-881, 2-15, 17-17, 19-19, 38-38, 56-61, 63-78, 81-83, 85-87, 91-99, 110-111, 114-114, 116-121, 124-125, 130-135, 145-149, 165-175, 177-177, 181-182, 185-185, 880-881, 993-998, 2-2, 16-16, 18-18, 20-37, 39-55, 62-62, 79-80, 84-84, 88-90, 100-109, 112-113, 115-115, 122-123, 126-129, 136-144, 150-164, 176-176, 178-180, 183-184, 186-186, 191-1039
  ✓ Structure prepared (using experimental coordinates)

[ErbB3 (HER3)]
  Fetching AlphaFold structure for P21860... ✓
  